# Locating gauge stations on MSC model grids



## Colaboratory requirements

If you are running this notebook as a demo in Google Colaboratory, you will need to uncomment the code in the next two cells. Some libraries are not available in Colaboratory, so we need to install them when we launch a Colab notebook session. Installing the following two libraries should only take a few seconds.

In [1]:
# !pip install owslib

In [ ]:
# !pip install hvplot

Next, we need to load the configuration file into this Colab notebook session. For example, let's assume we have a configuration file called <b>config.cfg</b> with the following format:
```
[Login]
Username = your_user_name 
Password = your_password
```

To load <b>config.cfg</b>, select the file folder icon on the left side of the screen. Hover over the folder named <b>sample_data</b> and select the three vertical dots to the right. Select <b>upload</b> and browse the file explorer to find <b>config.cfg</b>. Select <b>config.cfg</b> then click <b>Open</b>. You should notice <b>config.cfg</b> now appears in the list of files in <b>sample_data</b>. Hover over <b>config.cfg</b>, select the three vertical dots to the right, and then select <b>Copy path</b>. Paste the file path in the following cell to specify the path variable:

In [4]:
# example of path to config.cfg in Colab
path = '/content/sample_data/config.cfg'

## General requirements

In [5]:
# data
import warnings
import re
import time
import configparser
from datetime import datetime, timedelta
import xarray as xr 
import pandas as pd
import numpy as np
from shapely.geometry import Point

# web map services 
from owslib.wms import WebMapService
from owslib.wcs import WebCoverageService
from owslib.wcs import Authentication

# plotting
import holoviews
import hvplot.xarray

In [6]:
# if you are NOT running this in a Colab session, uncomment the following line to specify the path to the config file
#path = 'config.cfg'

In [8]:
# loading login information
config = configparser.ConfigParser()
config.read_file(open(path)) 

login = config['Login']

## Requesting latest DHPS predictions

In [15]:
layer_name = 'DHPS_1km_RiverDischarge'

In [16]:
# first querying the WMS for time metadata
wms = WebMapService(f'https://geo.weather.gc.ca/geomet?&SERVICE=WMS&LAYERS={layer_name}',
                    version='1.3.0',
                    auth=Authentication(username=login['Username'], password=login['Password']),
                    timeout=300)

In [17]:
first_datetime, last_datetime, datetime_interval = wms[layer_name].dimensions['time']['values'][0].split('/')
oldest_fcast, newest_fcast, issue_interval = wms[layer_name].dimensions['reference_time']['values'][0].split('/')

In [18]:
iso_format = "%Y-%m-%dT%H:%M:%SZ"

# convert dates to datetime objects
first = datetime.strptime(first_datetime, iso_format)
last = datetime.strptime(last_datetime, iso_format)

# remove anything that isn't a number from the datetime interval (time between forecasts)
intvl = int(re.sub(r'\D', '', datetime_interval))

# create a list of forecast datetimes (we will add these to the requested data)
fcasthrs = [first]
while first < last:
    first = first + timedelta(hours=intvl)
    fcasthrs.append(first)

# create a list of iso formatted forecast datetime strings (we will use these in the WCS requests)
fcasthrs_str = [datetime.strftime(hr, iso_format) for hr in fcasthrs]

Note: the next cell may take a few minutes to run because several requests are being made.

In [19]:
wcs = WebCoverageService(f'https://geo.weather.gc.ca/geomet?&SERVICE=WCS&COVERAGEID={layer_name}', 
                    auth=Authentication(username=login['Username'], password=login['Password']),
                    version='2.0.1',
                    timeout=300
                    )

# for each forecast hour, make a WCS request
arrys = []

for i, hr in enumerate(fcasthrs_str):
    response = wcs.getCoverage(identifier = [layer_name], 
                        format = 'image/netcdf', 
                        subsettingcrs = 'EPSG:4326', 
                        subsets = [('lat', 46.0, 52.0), ('lon', -96.0, -90.0)],
                        DIM_REFERENCE_TIME=newest_fcast, 
                        TIME=hr 
                       )
    
    # read into an xarray
    ds = xr.open_dataset(response.read()).load()
    
    # add the time metadata as a new dimension and coordinate
    ds = ds.expand_dims(time=[fcasthrs[i]])
    
    # append to list of xarrays
    arrys.append(ds)
    
fcasts = xr.concat([ds for ds in arrys], dim='time')

In [20]:
# checking to see that the time and lat/lon dimensions look sensible
fcasts.head()

<xarray.Dataset>
Dimensions:  (time: 5, lat: 5, lon: 5)
Coordinates:
  * time     (time) datetime64[ns] 2022-05-16T01:00:00 ... 2022-05-16T05:00:00
  * lat      (lat) float64 46.0 46.01 46.02 46.03 46.04
  * lon      (lon) float64 -96.0 -95.99 -95.98 -95.97 -95.96
Data variables:
    Band1    (time, lat, lon) float32 3.311e-05 0.0002619 ... 8.66e-05 0.0005255
Attributes:
    GDAL_TIFFTAG_RESOLUTIONUNIT:  2
    GDAL_TIFFTAG_XRESOLUTION:     72.0
    GDAL_TIFFTAG_YRESOLUTION:     72.0
    Conventions:                  CF-1.5
    GDAL:                         GDAL 3.1.3, released 2020/09/01
    history:                      Mon May 16 17:12:00 2022: GDAL CreateCopy( ...

## Finding stations on the DHPS model grid

In other tutorials, we've looked at the gridded DHPS data, but we may be more interested in the streamflow forecasts at specific stations. As an example, let's say we are interested in the flows at [Sturgeon River at McDougall Mills](https://wateroffice.ec.gc.ca/report/real_time_e.html?stn=05QA004). The following example demonstrates how we can extract the river discharge forecasts at our station of interest.


Note: the station latitutdes and longitudes in the models (DHPS/EHPS/WCPS) may not be exactly the same as the actual stations' latitudes and longitudes. In some cases, the modellers have had to slightly adjust the stations' latitudes/longitudes to insure the stations are properly aligned with the grid cells that contain their associated streams and minimize drainage area errors. This is a by-product of creating a gridded stream network. As a result, to find the stations of interest on the model grid we need to know what latitudes/longitudes the modellers have assigned to each station. In this example, Sturgeon River at McDougall Mills has an actual lat/lon of 50.167222, -91.540556 (in decimal degrees). In the model, Sturgeon River has a lat/lon of 50.1701, -91.5418. In addition, the model's station lat/lon may not align perfectly with the lat/lon axes of the data grid. To find the grid lat/lon we need to identify the closest grid point to the model's station lat/lon.

In [21]:
# Sturgeon River at McDougall Mills

# function to relate the CCMEP station location to its grid location
def find_stn_on_grid(latstn, lonstn):
    
    # find grid lat
    closest_lat_idx = (np.abs(fcasts.lat.data - latstn)).argmin()
    lat = fcasts.lat.data[closest_lat_idx]
    
    # find grid lon
    closest_lon_idx = (np.abs(fcasts.lon.data - lonstn)).argmin()
    lon = fcasts.lon.data[closest_lon_idx]
    
    return lat, lon

# CCMEP station locations
latst = 50.1701
lonst = -91.5418

# find the station on the grid
lat, lon = find_stn_on_grid(latst, lonst)

In [23]:
# compare the CCMEP station location and the grid station location
ll_info = f"CCMEP lat/lon ({str(latst)}, {str(lonst)}) | Grid lat/lon ({str(np.round(lat,4))}, {str(np.round(lon,4))})"
ll_info

'CCMEP lat/lon (50.1701, -91.5418) | Grid lat/lon (50.1708, -91.5458)'

In [11]:
# Uncomment the holoviews extension line below if running in a Colab notebook
# We need the following line in every cell that we use to generate a hvplot figure with Colab
# holoviews.extension('bokeh')

# plot the data
station = fcasts.sel(lat=lat, lon=lon)
station.hvplot(title='Sturgeon River at McDougall Mills | '+ll_info, ylabel='River Discharge (m3/s)', width=900)

:Curve   [time]   (Band1)